In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
headers = {'user-agent': 'Mozilla/5.0 (Macintosh Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36'}

In [ ]:
def find_text(items):
    data_text=[]
    texts=items[2].find("article",class_="article-content news-content")
    texts=texts.find_all("p")
    for i in range(len(texts)):
        data_text.append(texts[i].text.replace(u'\n',u'').replace(u'\xa0',u'').replace(u'\u3000',u'').strip())
    return str(data_text).replace('[','').replace(']','').replace("','",'').replace("'",'')
    
def find_reporter(items):
    items=items[2].find("dd",class_="publisher")
    reporter=items.text.strip()
    return reporter
def find_report_time(items):
    time=items[2].find("dd",class_="date").text.replace('年','-').replace('月','-').split('日')[0].replace('日','-')
    if '發布時間' in time:
        time=time.split(" ")[1]
    return time

In [ ]:
data_url=[]
data_title=[]
keywords=['颱風','天氣','歉收']#補上
search_type=['REL','LATEST']

In [ ]:
for k in keywords:
    for typ in search_type:
        url_origin='https://hub.line.me/search/'+k+'?module=news&sort='+typ+'&pageIndex='
        for i in range(1,2):#改至135
            try:
                url=url_origin+str(i)
                print(url)
                resp=requests.get(url,headers=headers)
                soup=BeautifulSoup(resp.text,'html5lib')
                items=soup.find('div',class_="searchToday-itemsContainer")
                items_title=items.find_all('h4')
                for i in items_title:
                    data_title.append(i.text.strip().replace(u'\u3000',u' '))
                items_url=items.find_all('a')
                for i in items_url:
                    data_url.append(i['href'])
            except:
                print("No items to crawl")
                break
                print("---------------")


In [ ]:
if len(data_title)==len(data_url):
    data=pd.DataFrame()
    data['title']=data_title
    data['url']=data_url
    data['website']='Line Today'
    print("make dataframe")
data.drop_duplicates(subset='url', keep='first', inplace=True)
data.reset_index(drop=True,inplace=True)

In [ ]:
data_reporter=[]
data_text=[]
data_time=[]


count=0
for i in data['url'][0:5]:
    try:
        resp=requests.get(i,headers=headers)
        #time.sleep(1)
        soup=BeautifulSoup(resp.text,"html5lib")
        items=soup.find_all("div",class_="inner")
        try:
            x=find_reporter(items)
            data_reporter.append(x)
            print(x)
        except:
            data_reporter.append("None")
        try:
            y=find_text(items)
            data_text.append(y)
            print(y)
        except:
            data_text.append("None")
        try:
            z=find_report_time(items)
            data_time.append(z)
            print(z)
        except:
        
            data_time.append("None")
    except:
        print("fail,全部為None")
        data_text.append("None")
        data_time.append("None")
        data_reporter.append("None")
    count+=1
    print("目前執行次數 %d" % count)
    print("目前共有幾筆文章資料 %d" % len(data_text))
    print("目前共有幾筆時間資料 %d" % len(data_time))

In [ ]:
data['reporter']=data_reporter
data['time']=data_time
data['text']=data_text

In [ ]:
text='2020年1月12日'
text.replace('年','-').replace('月','-').replace('日','')